This is an implementation of depolying CNN gensture reconition model to edge device (e.g. Sony Spresense)

Step Overview:
1. Conver Pytorch Model to Onnx Model
2. Conver Onnx Model to Keras Model
3. Conver Keras Model to quantization aware model
4. Retraining quantization aware model
5. Convert it to Tensorflow Lite Model
6. Using Edge Impuls to generate code for Sony Spresense

# Conver Pytorch Model to Onnx Model

In [1]:
import torch
import torchvision
from models.mobilenetv1 import MobilenetV1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dummy_input = torch.randn(1, 1, 8, 24, device=device)
model = MobilenetV1(ch_in=1, n_classes=8).to(device)
model.load_state_dict(torch.load(
    "pretrain_model/MobilenetV1_Param@29.29 k _MAC@233.1 KMac_Acc@95.346.pt"))
model.eval()

# print(model)
torch.onnx.export(model, dummy_input,
                  "pretrain_model/onnx_model/MobilenetV1.onnx", verbose=True)


/home/studenta/anaconda3/envs/sEMG/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


TypeError: __init__() got an unexpected keyword argument 'input_layer'

In [ ]:
import torch
import torchvision
from models.mobilenetv2_no_bn import MobileNetV2
dummy_input = torch.randn(1, 3, 300, 300, device="cuda")
# model = MobileNetV2(input_layer=1,num_classes=8,model_width=0.2).cuda()
# model.load_state_dict(torch.load("pretrain_model_test/MobilenetV2_Param@231.760K_MAC@1.855M_Acc@96.236.pt"), strict=False)
# model.eval()
model = torch.load('ssd.model')

# print(model)
torch.onnx.export(model, dummy_input, "ssd.onnx", verbose=True)

# Conver Onnx Model to Keras Model

In [ ]:
# Install library
%cd onnx2keras
!pip install -e .
%cd ..

In [ ]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf
import onnx

onnx_model = onnx.load("pretrain_model_test/onnx_model/MobilenetV2.onnx")
from onnx2keras import onnx_to_keras
model = onnx_to_keras(onnx_model, ['input.1'],name_policy='renumerate',verbose=False,change_ordering=True)
model.summary()

## Quantization aware training

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

from utils.ICE_lab_data_preprocessing import ICE_lab_data_preprocessing as utils

data,label,num_classes = utils().extra_data("data/Training_Trimmed")
from sklearn.model_selection import train_test_split

training_data, testing_data, training_label, testing_label = train_test_split(data, label, test_size=0.33, random_state=42)
train_data = tf.data.Dataset.from_tensor_slices((training_data, training_label))
test_data = tf.data.Dataset.from_tensor_slices((testing_data, testing_label))

training_data = training_data.reshape(-1,8,24,1)
testing_data = testing_data.reshape(-1,8,24,1)
training_data = utils().NormalizeData(training_data)
testing_data = utils().NormalizeData(testing_data)
q_aware_model.fit(training_data,training_label,
                  batch_size=1000, epochs=2)
_, q_aware_model_accuracy = q_aware_model.evaluate(
   testing_data, testing_label, batch_size=1000,verbose=True)
print('Quant test accuracy:', q_aware_model_accuracy)




In [ ]:
q_aware_model.save("pretrain_model_test/q_ware_model")

# Convert it to Tensorflow Lite Model

In [ ]:
import numpy as np
import os
def representative_dataset():
    data = np.load("representive_data.npy")
    for i in range(1):
        temp_data = data[i]
        temp_data = temp_data.reshape(1,8,24,1)
        yield [temp_data.astype(np.float32)]

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("pretrain_model_test/q_ware_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

# Save the model
with open("pretrain_model_test/tf_lite_model/mobilenetv2.tflite", 'wb') as f:
    f.write(tflite_model)

# Simulate Model Accuracy

In [ ]:
import numpy as np
import tensorflow as tf
import os

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="pretrain_model_test/tf_lite_model/mobilenetv2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)
for j in range(2,9):
    ori_input_data = np.load(f"representive_data{j}.npy")
    ori_input_data = ori_input_data.astype(np.int8)
    # ori_input_data = ori_input_data.reshape(-1,8,24,1)
    correct = 0
    print("Total Sample Size:",ori_input_data.shape[0])
    for i in range(ori_input_data.shape[0]):
        input_data = np.expand_dims(ori_input_data[i], 0)
        input_data = input_data.reshape(-1,8,24,1)
        interpreter.set_tensor(input_details[0]['index'], input_data)

        interpreter.invoke()

        # get_tensor() returns a copy of the tensor data
        # use tensor() in order to get a pointer to the tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        if np.argmax(output_data) == j-1:
            correct += 1
    print("Prediction Correct Size:",correct) #Total:30720
    print("Accuracy",round(correct/int(ori_input_data.shape[0]),2))